# Panomaly
"Understanding the network of user interaction anomalies"
### A Consulting Project for [Lazy Lantern](https://www.lazylantern.com)

## Donald W. Hoard, Ph.D.   
Data Science Fellow    
[Insight Data Science](https://insightfellows.com/data-science)   
Los Angeles - Jan 2020

An algorithm that uses a custom Hybrid Graph-Bayes model to identify linked anomalies in website/app user interactions. I built Panomaly in just 3 weeks during early January 2020 as a project for my [Insight Data Science](https://insightfellows.com/data-science).

Panomaly is intended as an add-on to [Lazy Lantern's](https://www.lazylantern.com) existing anomaly detection algorithm.
Panomaly is fully integrated with the raw event and processed anomaly databases, and can be quickly run at the end of every hour (following anomaly detection for that hour) to surface linked anomalies.
Panomaly produces a table of linked anomalies containing a single evaluation metric (Qpan) along with a verdict (True or False) indicating whether or not to surface the linkage to the client dashboard.
The verdict function (as well as the calculation of Qpan) can be easily modified as desired (or even set up to produce different results on an app-by-app basis).

### Nomenclature

* **event** = a single (raw) user action

* **metric** = a time series of the same event aggregated over all users

* **anomaly** = a metric flagged by the forecasting algorithm as outside the confidence interval for the current time interval

### Output
There are two dataframes created as output by Panomaly, which can be used as input to further code for surfacing the linked anomalies:

* ```pan_la_df```: The Linked Anomalies dataframe contains information about linked pairs of anomalies, including the individual and combined metrics used to evaluate the relative "strength" or "importance" of their linkage. For example: 

| _ | N1 | id_anom1 | id_metric1 | metric_name1 | N2 | id_anom2 | id_metric2 | metric_name2 | PbProb | PbNab | PfProb | Gmod1 | Gdeg1 | Gbtw1 | Gevc1 | Gavg1 | Gmod2 | Gdeg2 | Gbtw2 | Gevc2 | Gavg2 | pan_class | Qp | Qg | Qpan | Qcrit | Verdict |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 0 | 79 | 19021 | 4473 | Side Nav Clicked | 79 | 19021 | 4473 | Side Nav Clicked | 0.067568 | 5 | 0.004850 | 0 | 0.397059 | 0.264518 | 0.343285 | 0.334954 | 0 | 0.397059 | 0.264518 | 0.343285 | 0.334954 | 4 | 0.067741 | 0.473697 | 0.478516 | 0.0 | True |
| 1 | 79 | 19021 | 4473 | Side Nav Clicked | 149 | 19022 | 4470 | App Timing Metric | 0.158532 | 18 | 0.017459 | 0 | 0.397059 | 0.264518 | 0.343285 | 0.334954 | 2 | 0.514706 | 0.396241 | 0.462934 | 0.457960 | 2 | 0.159490 | -1.000000 | 0.105998 | 0.1 | True |
| 2 | 149 | 19022 | 4470 | App Timing Metric | 73 | 19024 | 4471 | First Contentful Paint | 0.583351 | 48 | 0.046557 | 2 | 0.514706 | 0.396241 | 0.462934 | 0.457960 | 2 | 0.176471 | 0.016147 | 0.203546 | 0.132054 | 3 | 0.585206 | 0.476620 | 0.754740 | 0.1 | True |
| 3 | 79 | 19021 | 4473 | Side Nav Clicked | 43 | 19023 | 4468 | First Paint | 0.000000 | 0 | 0.000000 | 0 | 0.397059 | 0.264518 | 0.343285 | 0.334954 | 2 | 0.147059 | 0.007143 | 0.191328 | 0.115177 | -1 | 0.000000 | 0.354203 | 0.000000 | 1.0 | False |
| 4 | 43 | 19023 | 4468 | First Paint | 149 | 19022 | 4470 | App Timing Metric | 0.221624 | 17 | 0.016489 | 2 | 0.147059 | 0.007143 | 0.191328 | 0.115177 | 2 | 0.514706 | 0.396241 | 0.462934 | 0.457960 | 3 | 0.222237 | 0.472222 | 0.521903 | 0.1 | True |
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... |


* ```pan_master_report_df```: The Master Report dataframe contains a list of grouped anomalies in different classes (described below) that have received a True verdict (surface to client dashboard). For example:

| _ | id_anom | id_metric | metric_name | pan_class | pan_class_tag |
| --- | --- | --- | --- | --- | --- |
| 0 | [19021] | [4473] | [Side Nav Clicked] | 4 | Pb & G (self) |
| 1 | [19022] | [4470] | [App Timing Metric] | 4 | Pb & G (self) |
| 3 | [19022, 19023, 19024] | [4470, 4468, 4471] | [App Timing Metric, First Paint, First Contentful Paint] | 3 | Pb & G |
| 4 | [19021, 19022] | [4473, 4470] | [Side Nav Clicked, App Timing Metric] | 2 | Pb only |



### About
For reference, this code version is ```panomaly-v14.0-offline```. This is the offline demonstration version of Panomaly that uses a subset of anonymized sample data rather than connecting on-the-fly to [Lazy Lantern's](https://www.lazylantern.com) proprietary databases.


### License
Permission to view and try the demo version of this code for personal or educational use is granted; however, copyright (including rights to modify and/or distribute) are solely held by the author (D. W. Hoard), with an unlimited license granted to Lazy Lantern, in perpetuity, to use, modify, and/or distribute the code.


---
## Basic external inputs
These are the necessary inputs that enable this code to define which subset of data to work on. They will have to be provided by the Python code that calls Panomaly.

* **pan_org_id** = Organization ID number (from organization table in postgreSQL dB)
* **pan_app_id** = Application ID number (from app table in postgreSQL DB)
* **pan_tstart** = start time of current interval (see sample format below)
* **pan_time_interval** = sampling window, in seconds (e.g., 3600 = 1 hour)

Manually set inputs used for testing are included in the next cell.
Change ```if True:``` to ```if False:``` to bypass the manual settings.


In [ ]:
#if False:
if True:
    pan_org_id = 3
    pan_app_id = 33
    
    pan_time_start = '2020-01-11 03:00:00' # sample time interval with many (13) anomalies

    # pan_time_interval is the 1 hour sampling window - this will generally not change
    pan_time_interval = 3600 # seconds


---
## Verbosity
Set the verbosity of this code.

```VERBOSE = False```: no diagnostic output (normal operation)    
```VERBOSE = True```: display diagnostic output


In [ ]:
#VERBOSE = False
VERBOSE = True


---
## Import general purpose packages used throughout the code
Packages used for specific tasks are imported later, before the relevant code blocks.

In [ ]:
import numpy as np
import pandas as pd


---
## Functions for communicating with databases
In the online version of Panomaly, the next cell defines functions used to interact with Lazy Lantern's databases. These have been removed to prevent access to Lazy Lantern's proprietary data. Instead, a subset of anonymized sample data will be used to demonstrate the functionality of Panomaly.

In [ ]:
# This cell is empty.

---
## Define other global functions

In [ ]:
# Convert a fractional probability (e.g., 0.15) to a 
# percentage (e.g., 15%) for pretty output.
def pan_pp(fx):
    return int(round((fx*100),0))


# This is a function used to combine individual metrics 
# on the way to producing the final metric Qpan.
# It is based on the geometric distance formula, and can 
# be thought of as defining a distance vector in the parameter 
# space of the two metrics being combined. It allows for some 
# combinations of x and y both being non-zero having a larger 
# "weight" than the case where one of x,y has the maximum value 
# and the other has the minimum value.
def combine_scores(x,y):
    return(np.sqrt(x**2 + y**2))


# Normalizes a list to the maximum of the list.
# Used during the calculation of Qpan.
def normalize_to_max(xlist):
    return (xlist)/np.max(xlist)


---
## The Verdict Function
This function evaluates the ```Qpan``` metric scores and returns a verdict (```True``` or ```False```) indicating whether the anomaly linkage should or should not be surfaced to the client dashboard, respectively. Several methods are encoded here; a particular method can be selected when the verdict function is called.

* **limit10**: This is the method I am using right now. It is very basic, and delivers a ```True``` verdict for ```Qpan >= 0.10```. Nonetheless, it seems to do a good job of surfacing the anomaly links that my manual inspections of the results suggest as the reliable ones.   


* **median**: This method compares the individual ```Qpan``` score to the median of all of the ```Qpan``` scores in a given ```pan_class``` (see ```pan_class``` designations below), and returns a verdict of True if ```Qpan >= median```.   


* **median1050**: This is similar to the **median** method, but sets the critical threshold to a value of 0.10 or 0.50 if the median is less or more than those values, respectivelty.   


* **mean**: Like the **median** method, but compares ```Qpan``` to the mean of all ```Qpan``` values in a ```pan_class```.   


* **all**: Returns a verdict of ```True``` for all values of ```Qpan```. This is used for ```pan_class = 4``` (self-linkages).


In [ ]:
def pan_verdict_function(x, y, method):
    if (method == 'limit10'): y_crit = 0.10
        
    if (method == 'median'): y_crit = np.median(y)
    
    if (method == 'median1050'): 
        y_crit = np.median(y)
        if y_crit < 0.1: y_crit = 0.1
        if y_crit > 0.5: y_crit = 0.5
    
    if (method == 'mean'): y_crit = np.mean(y)
    
    if (method == 'all'): y_crit = 0
    
    if x >= y_crit:
        return y_crit, True
    else:
        return y_crit, False


---
## Convert timestamps
This cell converts the start time ```pan_time_start``` into the correct format for use in queries, etc, and calculates the corresponding ```pan_time_stop```.


In [ ]:
from datetime import datetime, timedelta

# Converts string timestamp (e.g, "2020-01-01 00:00:40.958") to format allowing use of arithmatic operaters
def pan_ts_convert(tstamp):
    new_tstamp = -1
    ltstamp = len(tstamp)
    if ltstamp >= 23: 
        new_tstamp = datetime.strptime(str(tstamp).strip(), "%Y-%m-%d %H:%M:%S.%f")
    else:
        if ltstamp == 19:
            new_tstamp = datetime.strptime(str(tstamp).strip(), "%Y-%m-%d %H:%M:%S")
        else:
            if ltstamp == 10: 
                new_tstamp = datetime.strptime(str(tstamp).strip(), "%Y-%m-%d")
            else:
                print('timestamp conversion problem:', tstamp)
    return new_tstamp

pan_time_start = pan_ts_convert(pan_time_start)
pan_time_stop = pan_time_start + pd.Timedelta(seconds=pan_time_interval)

if VERBOSE: print('Time range:', pan_time_start, pan_time_stop)


---
## Load data from detected anomalies database
In the online version of Panomaly, only a minimum of relevant data is loaded from each table - just what is needed to complete the analysis for the current time interval.

In this offline demonstration verison of Panomaly, an anonymized subset of data is read from locally stored csv files, rather than pulling the data on-the-fly from the online databases.

In [ ]:
# Input table names
pan_table_list = ['app', 'metric', 'anomaly', 'anomaly_event']

# Input CSV sample data file names
pan_table_files = ['panomaly_'+table+'_sample_data.csv' for table in pan_table_list]

# Output dataframe names
pan_df_list = [table+'_df' for table in pan_table_list]

# The dataframes will be stored in a dictionary
pan_processed_dfs_dict = dict()

# Loop over the input/output names
for item_in, item_out in zip(pan_table_files, pan_df_list):
    if VERBOSE: print(item_in, item_out)
    pan_processed_dfs_dict[item_out] = pd.read_csv('./sample_data/'+item_in)    


---
## Clean the processed dataframes for missing values


In [ ]:
# Remove rows with missing id value
pan_processed_dfs_dict['anomaly_df'].dropna(subset=['id'], inplace=True)
pan_processed_dfs_dict['metric_df'].dropna(subset=['id'], inplace=True)
pan_processed_dfs_dict['app_df'].dropna(subset=['id'], inplace=True)

# Remove rows with missing cross-reference id value(s)
pan_processed_dfs_dict['anomaly_df'].dropna(subset=['metric_id'], inplace=True)
pan_processed_dfs_dict['metric_df'].dropna(subset=['app_id'], inplace=True)
pan_processed_dfs_dict['metric_df'].dropna(subset=['name'], inplace=True)
pan_processed_dfs_dict['app_df'].dropna(subset=['organization_id'], inplace=True)
pan_processed_dfs_dict['anomaly_event_df'].dropna(subset=['anomaly_id'], inplace=True)
pan_processed_dfs_dict['anomaly_event_df'].dropna(subset=['event'], inplace=True)
pan_processed_dfs_dict['anomaly_event_df'].dropna(subset=['occurred_at'], inplace=True)


---
## Load data from raw events database
In the online version of Panomaly, only a minimum of relevant data is loaded from each table - just what is needed to complete the analysis for the current time interval.

In this offline demonstration verison of Panomaly, an anonymized subset of data is read from locally stored csv files, rather than pulling the data on-the-fly from the online databases.

In [ ]:
pan_raw_df = pd.read_csv('./sample_data/panomaly_raw_events_sample_data.csv')  


---
## Clean the raw dataframe for missing values


In [ ]:
if VERBOSE: print('Raw data length before cleaning:', len(pan_raw_df))

# Remove rows with missing timestamp
pan_raw_df.dropna(subset=['timestamp'], inplace=True)

if VERBOSE: print('Raw data length after timestamp cleaning:', len(pan_raw_df))

# Remove rows with missing event
pan_raw_df.dropna(subset=['event'], inplace=True)

if VERBOSE: print('Raw data length after event cleaning:', len(pan_raw_df))

# Replace userId = None with userId = 'None'
# This conglomerates all unnamed users into a single user named 'None'
# User 'None' is actually useful, because it signals a user who clicked on the 
# signup page but did not register, so no userId exists for them.
pan_raw_df.loc[pan_raw_df['userId'].isnull(), 'userId'] = 'None'
# or comment the line above and uncomment the line below to delete all entries with unnamed users
#pan_raw_df.dropna(subset=['userId'], inplace=True)
                
if VERBOSE: print('Raw data length after userId cleaning:', len(pan_raw_df))


In [ ]:
# Set dataframe pretty print options for testing

# Reference(s):
# https://towardsdatascience.com/pretty-displaying-tricks-for-columnar-data-in-python-2fe3b3ed9b83
# https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/

if VERBOSE:
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.options.display.width = None
    pd.options.display.max_colwidth = None


---
## Build the master anomaly dataframe
This dataframe correlates all of the information in the other processed data (from the postgreSQL DB) dataframes to match up anomalies, metrics, apps, etc.

In [ ]:
# Match metric id with anomaly id
pan_ma_df = pan_processed_dfs_dict['anomaly_df'][['id', 'metric_id']].copy()
pan_ma_df.rename(columns={'id':'id_anom', 'metric_id':'id_metric'}, inplace=True)

# Match app id with metric id
pan_ma_df = pan_ma_df.join(pan_processed_dfs_dict['metric_df'].set_index('id').loc[:, ['app_id']], on='id_metric')
pan_ma_df.rename(columns={'app_id':'id_app'}, inplace=True)

# Match organization id and mongo_collection with app id
pan_ma_df = pan_ma_df.join(pan_processed_dfs_dict['app_df'].set_index('id').loc[:, ['organization_id']], on='id_app')
pan_ma_df.rename(columns={'organization_id':'id_org'}, inplace=True)

# Match anomaly start times with anomaly id
pan_df_test = pan_processed_dfs_dict['anomaly_event_df'][(pan_processed_dfs_dict['anomaly_event_df'].anomaly_id.isin(pan_ma_df.id_anom.values)) 
                & (pan_processed_dfs_dict['anomaly_event_df'].event == 'STARTED')]
pan_ma_df = pan_ma_df.merge(pan_df_test[['anomaly_id','occurred_at']], left_on='id_anom', right_on='anomaly_id')
pan_ma_df.rename(columns={'occurred_at':'tstart_anom'}, inplace=True)
pan_ma_df['tstart_anom'] = [pan_ts_convert(time) for time in pan_ma_df['tstart_anom']]
del pan_ma_df['anomaly_id']

# Import matching metric type
pan_ma_df = pan_ma_df.join(pan_processed_dfs_dict['metric_df'].set_index('id').loc[:, ['type']], on='id_metric')
pan_ma_df.rename(columns={'type':'metric_type'}, inplace=True)

# Import matching metric/event name - this one is important because 
# it is the only link to matching all of the information in this 
# dataframe to the corresponding events in the raw data.
pan_ma_df = pan_ma_df.join(pan_processed_dfs_dict['metric_df'].set_index('id').loc[:, ['name']], on='id_metric')
pan_ma_df.rename(columns={'name':'metric_name'}, inplace=True)


---
## Clean the master anomaly dataframe
This gets rid of all entries in the master anomaly dataframe that do not pertain to the selected app and org.

In [ ]:
if VERBOSE: print('pan_ma_df length before cleaning:', len(pan_ma_df))

# Remove missing values of id_app
pan_ma_df.dropna(subset=['id_app'], inplace=True)
pan_ma_df['id_app'] = pan_ma_df['id_app'].astype('int64')

# Remove missing values of id_org
pan_ma_df.dropna(subset=['id_org'], inplace=True)
pan_ma_df['id_org'] = pan_ma_df['id_org'].astype('int64')

if VERBOSE: print('pan_ma_df length before cleaning:', len(pan_ma_df))


---
## Sort and correlate events, anomalies, and metrics

In [ ]:
# This is the maximum value of metric_id, and will be used below to create new
# metric numbers for events that are in the raw data but not in the metric table.
pan_max_metric_id = max(pan_processed_dfs_dict['metric_df']['id'])

# Define the name of the timestamp column in the raw events dataframe.
pan_tcol_raw = 'timestamp'

# The event channel is not currently being constrained, but here is where you could set the constraint.
#pan_channel = 'client'

# Define the names of the timestamp columns in the master anomaly dataframe.
pan_tcol_ma_start  = 'tstart_anom'
pan_tcol_ma_stop   = 'tstop_anom'

# Select only the rows from the master anomaly dataframe relevant to the current app and org
pan_ma_selected_df = pan_ma_df[(pan_ma_df[pan_tcol_ma_start] >= pan_time_start) 
                        & (pan_ma_df[pan_tcol_ma_start] < pan_time_stop) 
                        & (pan_ma_df['id_org'] == pan_org_id) 
                        & (pan_ma_df['id_app'] == pan_app_id)].reset_index()

# Pull out lists for making comparisons
if len(pan_ma_selected_df.index):
    pan_anom_ids = pan_ma_selected_df['id_anom'].unique()

    pan_metric_ids = pan_ma_selected_df[(pan_ma_selected_df.id_anom.isin(pan_anom_ids))].id_metric.to_list()
    pan_nmetrics = len(pan_metric_ids)
        
    pan_metric_names = pan_ma_selected_df[(pan_ma_selected_df.id_anom.isin(pan_anom_ids))].metric_name.to_list()
    
    # This sorts all three lists in the same way so corresponding elements still match up in order
    pan_anom_ids, pan_metric_ids, pan_metric_names = zip(*sorted(zip(pan_anom_ids, pan_metric_ids, pan_metric_names)))

    pan_anom_ids = list(pan_anom_ids)
    pan_metric_ids = list(pan_metric_ids)
    pan_metric_names = list(pan_metric_names)

# Find number and IDs of unique users
pan_nusers = 0
if len(pan_raw_df.index):
    user_ids = pan_raw_df['userId'].unique()
    pan_nusers = len(user_ids)

    # Add metric ID to raw events dataframe
    if len(pan_ma_selected_df.index):
        pan_raw_df = pan_raw_df.join(pan_processed_dfs_dict['metric_df'].set_index('name').loc[:, ['id']], on='event')
        pan_raw_df.rename(columns={'id':'id_metric'}, inplace=True)
                
        # Deal with unknown metric IDs by assigning a new (unused) metric ID number to them
        # (i.e., events in pan_raw_df that are not listed in the metrics table with a corresponding id number)
        if pan_raw_df['id_metric'].isnull().values.any:
            pan_missing_metric_events = pan_raw_df[(pan_raw_df['id_metric'].isnull())].event
            pan_missing_metric_events_unique = set(pan_missing_metric_events)

            for item in pan_missing_metric_events_unique:
                # Start the new metric id numbers at the current maximum metric id plus one
                pan_max_metric_id += 1
                pan_raw_df.at[pan_raw_df['event'] == item, 'id_metric'] = pan_max_metric_id
        
        # Create list of metrics and transitions between metrics for each user
        pan_user_metrics_all = [] # list of metrics
        pan_user_transitions = [] # list of metric transitions
        
        for user in user_ids:                
            pan_user_metrics = (pan_raw_df[(pan_raw_df.userId == user)].id_metric.to_list())
            pan_user_metrics_all += pan_user_metrics
                        
            for i in range(len(pan_user_metrics)-1):
                pan_user_transitions.append([pan_user_metrics[i], pan_user_metrics[i+1]])
        
        pan_user_transitions_master = pan_user_transitions.copy()
        
        # Get rid of duplicates in metrics and transitions lists
        pan_user_metrics_all = list(set(pan_user_metrics_all)) 
        pan_user_transitions = [list(x) for x in set(tuple(x) for x in pan_user_transitions)]
    
        # Get rid of cases where both events/metrics/nodes are the same
        pan_user_transitions = [x for x in pan_user_transitions if len(set(x)) == 2]
    else:
        pan_user_metrics_all = []
        pan_user_transitions = []

if VERBOSE:
    print('Valid events:       ', len(pan_raw_df))
    print('Unique users:       ', pan_nusers)
    print('Anomalies:          ', len(pan_ma_selected_df))
    if len(pan_ma_selected_df.index):
        print('                    ', pan_anom_ids)
        print('Metrics:            ', pan_nmetrics)
        print('                    ', pan_metric_ids)
        print('                    ', pan_metric_names)


---
## Check for anomalies and compile anomalies with zero user events
If no anomalies are present, then we're done - the algorithm passes through to the end without trying to do more. This action is controlled by the value of ```panomaly_status```:

* ```True``` = anomalies are present, continue
* ```False``` = no anomalies are present, do not continue

In some cases, anomalies are present without any corresponding user events (this is probably why they were anomalous), so I also deal with that case here. These "zero event anomalies" might be present with or without other anomalies.

In [ ]:
# Initialize list for storing the IDs of zero event anomalies
pan_anom_zero_events = [] 

# Set panomaly_status
if not len(pan_ma_selected_df.index): 
    panomaly_status = False
    if VERBOSE: print('\n*** WARNING! NO ANOMALIES! NO ADDITIONAL ANALYSIS WILL BE PERFORMED! ***\n')
else: 
    if not len(pan_raw_df.index):
        panomaly_status = False
    else: panomaly_status = True
    
    # Compile the zero event anomalies
    for i, metric_name in enumerate(pan_metric_names):
        if (metric_name not in pan_raw_df.event.to_string()):
            pan_anom_zero_events.append([pan_anom_ids[i], pan_metric_ids[i], metric_name])
    if pan_anom_zero_events:
        if VERBOSE:
            print('======================================')
            print('The following metrics are probably flagged as anomalous becuase')
            print('they have no user event triggers in the current interval:')
            print(*pan_anom_zero_events, sep='\n')


---
## Build the linked anomalies dataframe
This will be the main dataframe for storing diagnostic information about potentially linked anomalies.

In [ ]:
import itertools

pan_la_df = []

if panomaly_status:
    temp_data_dict = {'N1': [], 'id_anom1':[], 'id_metric1':[], 'metric_name1':[], 'N2':[], 'id_anom2':[], 'id_metric2':[], 'metric_name2':[]}
    for (a1, m1, n1), (a2, m2, n2) in itertools.product(zip(pan_anom_ids, pan_metric_ids, pan_metric_names), zip(pan_anom_ids, pan_metric_ids, pan_metric_names)):
        # These values are transferred from pan_ma_df
        temp_data_dict['id_anom1'].append(a1)
        temp_data_dict['id_metric1'].append(m1)
        temp_data_dict['metric_name1'].append(n1)
        temp_data_dict['id_anom2'].append(a2)
        temp_data_dict['id_metric2'].append(m2)
        temp_data_dict['metric_name2'].append(n2)

        # N1 and N2 are the number of events for each metric
        temp_data_dict['N1'].append(len(pan_raw_df[(pan_raw_df['id_metric'] == m1)]))
        temp_data_dict['N2'].append(len(pan_raw_df[(pan_raw_df['id_metric'] == m2)]))
    
    # Create the dataframe
    pan_la_df = pd.DataFrame(temp_data_dict)

    # Probability columns:
    #     PbProb = Bayesian probability
    #     PbNab = number of transitions from metric "a" to metric "b"
    #     PfProb = frequentist probability
    pan_la_df['PbProb'] = list(len(pan_la_df)*[0])
    pan_la_df['PbNab'] = list(len(pan_la_df)*[0])
    pan_la_df['PfProb'] = list(len(pan_la_df)*[0])

    # Graph columns (containing graph metrics):
    #     Gmod* = modularity class (nodes that share an overdensity of edges)
    #     Gdeg* = degree (number of edges for a given node, as a fraction of the total number of nodes)
    #     Gbtw* = betweennes (number of shortest paths between two nodes that pass through the given node)
    #     Gevc* = eigenvector centrality (importance of a node measured by the number of "important" nodes connected to it)
    pan_la_df['Gmod1'] = list(len(pan_la_df)*[-1])
    pan_la_df['Gdeg1'] = list(len(pan_la_df)*[0])
    pan_la_df['Gbtw1'] = list(len(pan_la_df)*[0])
    pan_la_df['Gevc1'] = list(len(pan_la_df)*[0])
    pan_la_df['Gavg1'] = list(len(pan_la_df)*[0])
    pan_la_df['Gmod2'] = list(len(pan_la_df)*[-1])
    pan_la_df['Gdeg2'] = list(len(pan_la_df)*[0])
    pan_la_df['Gbtw2'] = list(len(pan_la_df)*[0])
    pan_la_df['Gevc2'] = list(len(pan_la_df)*[0])
    pan_la_df['Gavg2'] = list(len(pan_la_df)*[0])

    # Transition class (see below for class descriptions)
    pan_la_df['pan_class'] = list(len(pan_la_df)*[-1])

    # Panomaly metrics
    #    Qp = combined probability metric
    #    Qg = combined graph metric
    #    Qpan = final evaluation metric (combination of Qp and Qg)
    #    Qcrit = critical threshhold for Qpan used to determine the verdict
    pan_la_df['Qp'] = list(len(pan_la_df)*[0])
    pan_la_df['Qg'] = list(len(pan_la_df)*[0])
    pan_la_df['Qpan'] = list(len(pan_la_df)*[0])
    pan_la_df['Qcrit'] = list(len(pan_la_df)*[1])

    # Evaluation of whether or not the anomaly linkage should be surfaced to the client dashboard
    pan_la_df['Verdict'] = list(len(pan_la_df)*[False])


---
## Bayesian Probability analysis
Bayesian probability incorporates prior knowledge about the attributes of a system to inform the calculation of probabilities of a given event occurring based on knowing what previous event occurred.

It is based on Bayes's Theorem:

```P(A|B) = P(A)P(B|A)/P(B)```

where 

* ```P(A|B)``` = the probability that event A occurs after event B - this is what we want to find.


* ```P(A)``` = the probability that event A occurs irrespective of event B.


* ```P(B)``` = the probability that event B occurs irrespective of event A.


* ```P(B|A)``` = the probability that event B occurred after event A based on the data.

In this case, we can define the events as

**A**: metric2 = m2 (i.e., the 2nd metric is m2)    

**B**: metric1 = m1 (i.e., the 1st metric is m1)

and the probabilities are

* ```P(A)= NA / Ntotal``` (# cases with metric2 = m2 / # all cases)


* ```P(B)= NB / Ntotal``` (# cases with metric1 = m1 / # all cases)


* ```P(B|A) = N(m1,m2) / NA``` (the probability that metric1 = m1 when metric2 = m2)


In [ ]:
if panomaly_status:
    
    from collections import Counter # This provides a dictionary subclass for counting hashable objects

    # Count the number of each possible transitions between events (nodes)
    pan_bayes_counter = Counter(tuple(x) for x in iter(pan_user_transitions_master))
    pan_Ntotal = len(pan_user_transitions_master)

    if VERBOSE:
        print('All anomaly ids:', pan_anom_ids)
        print('All anomaly metrics:', pan_metric_ids)
        print('Total event transitions:', pan_Ntotal)
        # N might be less than expected because some users only initiate a total of 1 event (i.e., no transitions between events)
        print('Total unique event transitions:', len(pan_bayes_counter))
        pan_Pcrit = 0.01 # critical probability threshold for reporting a potential linkage
        print('Probability reporting threshold >= '+str(pan_pp(pan_Pcrit)).strip()+'%')

    pan_set_metric_ids = set(pan_metric_ids) # use this to get probabilities for just unique anomaly metric transitions
    #pan_set_metric_ids = pan_user_metrics_all # use this to get probabilities for all unique metric transitions
    
    for m1, m2 in itertools.product(pan_set_metric_ids, pan_set_metric_ids):
        pan_A = sum([pan_bayes_counter[item] for item in pan_bayes_counter if item[1] == m2])
        pan_B = sum([pan_bayes_counter[item] for item in pan_bayes_counter if item[0] == m1])
        pan_BA = (pan_bayes_counter[(m1, m2)])

        if (pan_Ntotal > 0) & (pan_A > 0):
            pan_PA  = pan_A/pan_Ntotal
            pan_PBA = pan_BA/pan_A
            pan_PB  = pan_B/pan_Ntotal

            if (pan_PB > 0): 
                pan_PAB = pan_PA*pan_PBA/pan_PB
                    
                if VERBOSE: 
                    if (pan_PAB >= pan_Pcrit): print(m1,'+',m2,': P(%) =',pan_pp(pan_PAB),' [N_AB = '+str(pan_BA)+']')

                if (pan_PAB > 0):
                    PbProb = float(pan_PAB)
                    pan_la_df.at[((pan_la_df['id_metric1'] == m1) & (pan_la_df['id_metric2'] == m2)), 'PbProb'] = [PbProb]
                        
                    PbNab = int(pan_BA)
                    pan_la_df.at[((pan_la_df['id_metric1'] == m1) & (pan_la_df['id_metric2'] == m2)), 'PbNab'] = [PbNab]
                        
                    PfProb = float(pan_BA/pan_Ntotal)
                    pan_la_df.at[((pan_la_df['id_metric1'] == m1) & (pan_la_df['id_metric2'] == m2)), 'PfProb'] = [PfProb]


---
## Network Graph analysis
I apply fairly  standard network graph analysis techniques, using the Python module ```networkx```.

Networks consist of nodes (the events) and edges (the transitions between events). I am using an undirected, weighted network. Undirected means that transitions between events are allowed to go in either direction (i.e., A to B and B to A). Weighted means that the edges are weighted by the total number of corresponding transitions, to allow for the "popularity" or "importance" of certain sequences of events (as evidenced by the actions of the users).

Note: in a fresh installation of Anaconda, the following additional package must be installed to provide some fo the graph analysis functionality.

```pip install python-louvain```

Note: instances of ```G.node[]``` have been changed to ```G.nodes[]``` for compatibility with networkx 2.4+.


In [ ]:
# These commands should really be at the top of the next cell, but they
# are here because of a known bug between matplotlib and Jupyter Notebooks 
# that makes displaying a figure fail during the first run after starting 
# or restarting the kernel if the matplotlib import is done in the same 
# cell as the plot.

# Reference(s):
#     https://github.com/jupyter/notebook/issues/3670

if panomaly_status:
    if VERBOSE:
        import time # used for establishing a random seed for plotting
        import matplotlib.pyplot as plt # for making graph plot


In [ ]:
if panomaly_status:
    import community # This is the python-louvain package
    import networkx as nx
    from operator import itemgetter

    pan_nodes = pan_user_metrics_all.copy() # These are the events
    pan_Nnodes = len(pan_nodes)    
    pan_edges = pan_user_transitions.copy() # These are the transitions between events
    
    # Get rid of transitions where both nodes are the same (i.e., not really a transition) 
    # but keep duplicate transitions.
    pan_user_transitions_wts = [x for x in pan_user_transitions_master if len(set(x)) == 2]
    pan_edge_wts = Counter(tuple(x) for x in iter(pan_user_transitions_wts))
        
    # Create list of edge weights
    # pan_wedges_friends = weight is the number of observed transitions from all users; appropriate 
    #     for finding modularity & eigencentrality, because it implies the strength of "friendship" between nodes
    # pan_wedges_enemies = weights reversed from pan_wedges_friends; appropriate for finding 
    #     betweenness, which assumes high weight indicates a "high cost" (i.e., "bad") transition
    # e.g., see Santiago Segarra and Alejandro Ribeiro, "Stability and Continuity of Centrality Measures in 
    #     Weighted Graphs", https://arxiv.org/pdf/1410.5119.pdf; M. E. J. Newman, PHYSICAL REVIEW E 70, 056131 
    #     (2004), "Analysis of weighted networks", http://www.uvm.edu/pdodds/files/papers/others/2004/newman2004d.pdf
    pan_wedges_friends = []    
    [pan_wedges_friends.append(tuple([edge[0], edge[1], pan_edge_wts[tuple(edge)]])) for edge in pan_edges]
    #[pan_wedges_friends.append(tuple([edge[0], edge[1], 1])) for edge in pan_edges]
    wt_min = min([item[2] for item in pan_wedges_friends])
    wt_max = max([item[2] for item in pan_wedges_friends])
    pan_wedges_enemies = []    
    [pan_wedges_enemies.append(tuple([edge[0], edge[1], wt_max-pan_edge_wts[tuple(edge)]+wt_min])) for edge in pan_edges]
    #pan_wedges_enemies = pan_wedges_friends.copy()    

    # Initialize empty graph
    G = nx.Graph()
    # Populate the nodes
    G.add_nodes_from(pan_nodes)
    # Populate the weighted edges
    G.add_weighted_edges_from(pan_wedges_friends, weight='weight_friends') 
    G.add_weighted_edges_from(pan_wedges_enemies, weight='weight_enemies') 

    # Calculate graph metrics...
    
    # DEGREE = the number of connections (edges) a node has to other nodes, as a fraction 
    # of the total number of nodes in the network
    pan_degree_dict = dict(G.degree(G.nodes()))
    nx.set_node_attributes(G, pan_degree_dict, 'degree') # assign as node attribute

    # BETWEENNESS CENTRALITY = the number of times a node lies on the shortest path 
    # between two other nodes.
    pan_betweenness_dict = nx.betweenness_centrality(G, weight='weight_enemies') # obtain betweenness centrality
    nx.set_node_attributes(G, pan_betweenness_dict, 'betweenness') # assign as node attribute
    
    for i in pan_metric_ids:
        if i in pan_degree_dict:
            pan_la_df.at[((pan_la_df['id_metric1'] == i)), 'Gdeg1'] = pan_degree_dict[i]/pan_Nnodes
            pan_la_df.at[((pan_la_df['id_metric2'] == i)), 'Gdeg2'] = pan_degree_dict[i]/pan_Nnodes

        if i in pan_betweenness_dict:
            pan_la_df.at[((pan_la_df['id_metric1'] == i)), 'Gbtw1'] = pan_betweenness_dict[i]
            pan_la_df.at[((pan_la_df['id_metric2'] == i)), 'Gbtw2'] = pan_betweenness_dict[i]        
    
    # MODULARITY CLASSES have dense connections between the nodes within the modularity 
    # classes but sparse connections between nodes in different modules.
    
    # The following try/except block is here because of an early problem caused by uncleaned 
    # data (events with unnumbered metrics) which has now been fixed. So, the exception  
    # should never be reached here - but why not keep it anyway, just in case...
    try:
        pan_modularity_classes = community.best_partition(G, weight='weight_friends')
    except KeyError:
        print('KeyError in community.best_partition - canceling remainder of Graph analysis')
        panomaly_status = False
        
        
    # Continued after the following VERBOSE block...
    
        
    # Lots of diagnostic information will be displayed in verbose mode. None of this 
    # is necessary for the anomaly linkage analysis. The necessary stuff is outside 
    # the VERBOSE block.
    if VERBOSE:
        # Basic network information
        print(nx.info(G))
        
        # Network density is the number of edges (transitions between nodes) that 
        # exist divided by the total number of all edges that could exist (regardless 
        # of whether or not they do).
        pan_density = nx.density(G)
        print("Network density:", pan_density)
        print('--------------------------------')

        # Graph components are (sub)sets of nodes that have no edges connecting them. 
        # Often a graph will have only one component, with at least one edge connecting 
        # all nodes. Such a graph is also called "connected". But sometimes two or more 
        # groups of "disconnected" nodes are present. In the user interactions scenario, 
        # I see some graphs with up to several disconnected components, typically 
        # containing only 1 or 2 nodes. These likely correspond to users who interacted  
        # only shortly with the app and performed a couple of infrequently used actions.       
        # 
        # The following statement returns False for a disconnected graph:
        pan_single_component = nx.is_connected(G)
        print('Is this a single-component network?', pan_single_component)
        
        # Network diameter is the longest path (number of edges) connecting two nodes.
        if (pan_single_component == True): 
            print('Single Component Network Diamter = ', nx.diameter(G))

        # Diameter cannot be directly calculated for a disconnected graph, so the 
        # diameter of the largest component is used instead.
        # nx.connected_components returns a list of the graph components, and
        # max() finds the largest one.
        pan_components = nx.connected_components(G)
        pan_largest_component = max(pan_components, key=len)

        # Create a subgraph of just the largest component and return its diameter.
        pan_subgraph = G.subgraph(pan_largest_component)
        print("Network diameter of largest component:", nx.diameter(pan_subgraph))
        print('--------------------------------')
        
        pan_sorted_degree = sorted(pan_degree_dict.items(), key=itemgetter(1), reverse=True)
        print("Top 20 nodes by degree:")
        [print(d) for d in pan_sorted_degree[:20]]
        print('--------------------------------')
        
        pan_sorted_betweenness = sorted(pan_betweenness_dict.items(), key=itemgetter(1), reverse=True)
        print("Top 20 nodes by betweenness centrality:")
        [print(b) for b in pan_sorted_betweenness[:20]]
        print('--------------------------------')
        
        # Get the top 20 nodes by betweenness as a list
        pan_top_betweenness = pan_sorted_betweenness[:20]
        # Then find and print their degree
        for tb in pan_top_betweenness: # Loop through pan_top_betweenness
            pan_degree = pan_degree_dict[tb[0]] # Use pan_degree_dict to get a node's degree
            print("Name:", tb[0], "| Betweenness Centrality:", tb[1], "| Degree:", pan_degree)
        print('--------------------------------')
        
        if panomaly_status:
            pan_global_modularity = community.modularity(pan_modularity_classes, G)
            print("Global Modularity =", pan_global_modularity)
            print('--------------------------------')
            print("Modularity Class Sorted by Eigenvector Centrality:")

            
    if panomaly_status:
        nx.set_node_attributes(G, pan_modularity_classes, 'modularity') # assign as node attribute
    
        # EIGENVECTOR CENTRALITY = a measure of the influence of a node in a network; a high 
        # eigenvector centrality means that a node is connected to many nodes who themselves 
        # have high scores.   
        pan_eigenvector_dict = nx.eigenvector_centrality(G, weight='weight_friends') # obtain eigenvector centrality
        nx.set_node_attributes(G, pan_eigenvector_dict, 'eigenvector') # assign as node attribute
   
        # Sort nodes into modularity classes
        pan_modularity_dict = {} # Create a new, empty dictionary
        for k,v in pan_modularity_classes.items(): # Loop through the modularity classes dictionary
            if v not in pan_modularity_dict:
                pan_modularity_dict[v] = [k] # Add a new key for a modularity class that hasn't been seen before
            else:
                pan_modularity_dict[v].append(k) # Append a name to the list for a modularity class that has already been seen

                
        if VERBOSE:
            # pan_G_modularity_class_dict is a remnant of early code development. 
            # It is no longer used in the final algorithm, but it contains a useful compilation 
            # of information and is a little difficult to populate the dictionary, so I have 
            # left the code in. It only runs in verbose mode.
            pan_G_modularity_class_dict = {'Gmodclass':[], 'GNmodclass':[], 'GmodMembers':[], 'Gmodanoms':[], 'Ganomevents':[]}
                # Gmodclass = modularity class identifier (integer number)
                # GNmodclass = number of nodes in modularity class
                # GmodMembers = list of nodes in modularity class
                # Gmodanoms = list of anomalous metrics in modularity class
                # Ganomevents = list of anomalous metric event names in modularity class
        
        
        for k,v in pan_modularity_dict.items(): # Loop through the new modularity classes dictionary
            # Cannot ignore single-member modularity classes because a disconnected anomalous 
            # metric would not be surfaced as linked to itself (if appropropriate).
            if len(v) > 0:   
                # Get a list of just the nodes in that class
                pan_classN = [n for n in G.nodes() if G.nodes[n]['modularity'] == k]

                # Create a dictionary of the eigenvector centralities of those nodes
                pan_classN_eigenvector = {n:G.nodes[n]['eigenvector'] for n in pan_classN}

                # Sort the eigencentrality dictionary
                pan_classN_sorted_by_eigenvector = sorted(pan_classN_eigenvector.items(), key=itemgetter(1), reverse=True)

                # Add results to the linked anomalies dataframe
                for node in pan_classN_sorted_by_eigenvector:
                    pan_la_df.at[((pan_la_df['id_metric1'] == node[0])), 'Gmod1'] = k
                    pan_la_df.at[((pan_la_df['id_metric2'] == node[0])), 'Gmod2'] = k

                    pan_la_df.at[((pan_la_df['id_metric1'] == node[0])), 'Gevc1'] = node[1]
                    pan_la_df.at[((pan_la_df['id_metric2'] == node[0])), 'Gevc2'] = node[1]
                    
                    if VERBOSE: print('Class:', k, '| Name:', node[0], '| Eigenvector Centrality:', node[1])

                        
                if VERBOSE:
                    pan_G_modularity_class_dict['Gmodclass'].append(str(k))
                    pan_G_modularity_class_dict['GNmodclass'].append(str(len(v)))
                    pan_G_modularity_class_dict['GmodMembers'].append(v)

                    # This is the list of anomalies in the current modularity class
                    pan_anom_metrics = list(set(pan_metric_ids).intersection(v))
                    pan_G_modularity_class_dict['Gmodanoms'].append(pan_anom_metrics)
                
                    print('Modularity Class Metrics:', pan_classN)
                    if pan_anom_metrics: 
                        print('Included Anomaly Metrics:', pan_anom_metrics)

                    # Pull out the event names of anomalous metrics in modularity classes
                    pan_metric_idx = [pan_metric_ids.index(x) for x in pan_anom_metrics]
                    if pan_metric_idx: 
                        pan_metric_names_output = [pan_metric_names[x] for x in pan_metric_idx]
                        print('                         ', pan_metric_names_output)                        
                    else:
                        pan_metric_names_output = []

                    pan_G_modularity_class_dict['Ganomevents'].append(pan_metric_names_output)
                    print('')        

        if VERBOSE:
            pan_G_modularity_class_df = pd.DataFrame(pan_G_modularity_class_dict)
            print('--------------------------------')    
            print('All anomaly metrics:', pan_metric_ids)

            #### PLOT GRAPH
            print('--------------------------------')    
            print('\nNetwork Graph.')
            print('Nodes are size-coded by degree and color-coded by modularity class.')
            print('Anomalies are shown as red diamonds (with metric_ids labeled).')
            
            # Prepare node labels for plotting the Graph
            pan_node_labels = {}    
            for node in G.nodes():
                if node in pan_metric_ids:
                    # Set the node name as the key and the label as its value 
                    pan_node_labels[node] = node

            # This is the current sytem time to use as a seed for the plot
            pan_draw_seed = round(time.time()) # has to be an integer between 1 and (2**32 - 1)
            # The spring plot imagines the edges as springs with spring constant kspring. 
            # The network is spun around its center and "stretches" the nodes apart 
            # to make the network legible.
            kspring = 7.0/np.sqrt(pan_Nnodes)
            pos = nx.spring_layout(G, k=kspring, seed=pan_draw_seed)
            
            # If the graph analysis happened, color-code the nodes by modularity class; otherwise, use blue
            if panomaly_status:
                pan_node_color = [G.nodes[value]['modularity'] for value in G.nodes]
            else:
                pan_node_color = 'b'

            # Size-code the nodes by their degree
            pan_node_size0 = [G.nodes[value]['degree'] for value in G.nodes]
            pan_max_node_size = max(pan_node_size0)
            pan_min_node_size = 15
            pan_node_size_scale_factor = 80
            pan_node_size = [(pan_min_node_size + pan_node_size_scale_factor*(value/pan_max_node_size)) for value in pan_node_size0]

            # Set up the figure
            f = plt.figure(figsize=[6,6], dpi=600)
            ax = f.add_subplot(1, 1, 1)

            # Plot the network
            nx.draw(G, pos, ax=ax, cmap=plt.get_cmap('viridis'), vmin=min(pan_node_color), vmax=max(pan_node_color), node_color=pan_node_color, node_size=pan_node_size, with_labels=False, width=0.25)

            # Overplot the anomalies as large red diamonds
            for metric_id in pan_node_labels:
                value = G.nodes[metric_id]['degree']
                pan_node_size = (pan_min_node_size + pan_node_size_scale_factor*(value/pan_max_node_size))
                nx.draw_networkx_nodes(G, pos, node_size=pan_node_size, node_shape='D', nodelist=[metric_id], node_color='r')

            # Label just the anomalies with their metric ID
            nx.draw_networkx_labels(G, pos, pan_node_labels, font_size=4, font_color='k')

            # Uncomment the next line to save the network graph figure to disk
            #plt.savefig('figures/graph'+str(pan_draw_seed)+'.png') # save as png

            plt.show() # display


---
## Combine probabilities
Now that the Graph analysis is done, one more thing to do for the probability analysis: combine the probabilities for mirror image transitions (A to B and B to A) because we don't care in which direction anomalies are linked, just that they are linked. It's a "this or that" situation, so the probabilities for the two cases can just be added together.


In [ ]:
if panomaly_status:
    # Going to work in a copy of the linked anomalies dataframe, then copy it back to the original frame at the end
    pan_la_temp_df = pan_la_df.copy()

    # Loop through the dataframe and find rows that have the same transition
    pan_used_idx = []
    for m1, m2 in itertools.product(pan_set_metric_ids, pan_set_metric_ids):
        if m1 != m2:
            pan_temp_df0 = pan_la_df[(pan_la_df.id_metric1 == m1) & (pan_la_df.id_metric2 == m2)].copy()
            pan_temp_df1 = pan_la_df[(pan_la_df.id_metric1 == m2) & (pan_la_df.id_metric2 == m1)].copy()
                
            if (len(pan_temp_df0.index) > 0):
                if (not pan_temp_df0.index.isin(pan_used_idx)):
                    pan_used_idx.append(pan_temp_df0.index[0])
                    if (len(pan_temp_df1.index) > 0): 
                        # Record the indices so we don't try to operate on the same row(s) twice
                        pan_used_idx.append(pan_temp_df1.index[0])
                        # Combine the probabilities
                        pan_Psum = (pan_temp_df0.iloc[0]['PbProb'] + pan_temp_df1.iloc[0]['PbProb'])
                        pan_Nsum = (pan_temp_df0.iloc[0]['PbNab'] + pan_temp_df1.iloc[0]['PbNab'])
                        pan_Pfsum = (pan_temp_df0.iloc[0]['PfProb'] + pan_temp_df1.iloc[0]['PfProb'])
            
                        # Populate the revised row to the dataframe
                        pan_la_temp_df.at[pan_temp_df0.index[0], 'PbProb'] = pan_Psum
                        pan_la_temp_df.at[pan_temp_df0.index[0], 'PbNab'] = pan_Nsum
                        pan_la_temp_df.at[pan_temp_df0.index[0], 'PfProb'] = pan_Pfsum

                        # Get rid of the now redundant row
                        pan_la_temp_df = pan_la_temp_df[pan_la_temp_df.index != pan_temp_df1.index[0]]
    
    # Copy the working dataframe back to the original dataframe
    pan_la_df = pan_la_temp_df.copy()


---
## Housekeeping
Account for the (uninteresting) case when there is only 1 anomaly and it is not linked to itself.


In [ ]:
if panomaly_status:
    if pan_nmetrics == 1:
        if pan_la_df[pan_la_df['id_metric1'] == pan_metric_ids[0]].PbNab.to_list()[0] < 1: panomaly_status = False


---
## Combine metrics
Combine the probability and graph metrics into a single metric for each analysis. I used Latent Factor Analysis to confirm that all of the individual metrics are related by a common underlying (but unknown) relationship (positive correlation), which makes it appropriate to combine them.

In [ ]:
if panomaly_status: 
    # The probability metric Qp is created by combining the Bayesian probability 
    # and the frequentist probability using the combine_scores method. This 
    # downweights cases where the Bayesian probability is dependent on a small 
    # number of actual instances.
    Qp = combine_scores((pan_la_df['PbProb']), (pan_la_df['PfProb']))
    Qp_max = max(Qp)
    pan_la_df['Qp'] = Qp

    # The graph metric Qg is created by averaging the 3 graph metrics for each 
    # node in a transition, then using the combine_scores method on the two 
    # average values.
    pan_avg_divisor = 3
    pan_la_df['Gavg1'] = (pan_la_df['Gdeg1'] + pan_la_df['Gbtw1'] + pan_la_df['Gevc1']) / pan_avg_divisor
    pan_la_df['Gavg2'] = (pan_la_df['Gdeg2'] + pan_la_df['Gbtw2'] + pan_la_df['Gevc2']) / pan_avg_divisor

    Qg = combine_scores(pan_la_df['Gavg1'], pan_la_df['Gavg2'])
    Qg_max = max(Qg)
    pan_la_df['Qg'] = Qg


---
## Sort linked anomalies into classes
The linked anomalies are assigned to a class based on the following criteria:

* **Class 4** = Anomalies linked to themselves, resulting from users that triggered the same event multiple times in succession. These also have high Qp and Qg values.


* **Class 3** = Linked anomalies with agreement between the graph and probability analyses (high Qp and Qg). These are the most reliable linkages.


* **Class 2** = Anomalies linked by a high Qp value only (i.e., they are not members of the same modularity class, so have Qg = 0). Less reliable linkages.


* **Class 1** = Anomalies linked by the graph analysis only (i.e., they are members of the same modularity class). There were no actual user events linking these anomalies (PbNab = 0). Exercise caution: the graph analysis predicts that these anomalies would be linked, but we have no data showing that users transitioned from one event to the other.


* **Class 0** = Metrics that are anomalous during the current time interval because they have no user events (i.e., no user clicked on this event during this hour, but users did in past hours).


In [ ]:
pan_classes = [4, 3, 2, 1]
pan_class_tags = {0:'No user events', 1:'G only', 2:'Pb only', 3:'Pb & G', 4:'Pb & G (self)'}

if panomaly_status: 
    pan_la_df.loc[(pan_la_df['id_metric1'] != pan_la_df['id_metric2']) & (pan_la_df['PbNab'] < 1) & ((pan_la_df['Gmod1'] >= 0) & (pan_la_df['Gmod2'] >= 0)) & (pan_la_df['Gmod1'] == pan_la_df['Gmod2']), 'pan_class'] = 1
    pan_la_df.loc[(pan_la_df['id_metric1'] != pan_la_df['id_metric2']) & (pan_la_df['PbNab'] > 0) & ((pan_la_df['Gmod1'] >= 0) & (pan_la_df['Gmod2'] >= 0)) & (pan_la_df['Gmod1'] != pan_la_df['Gmod2']), 'pan_class'] = 2
    pan_la_df.loc[(pan_la_df['id_metric1'] != pan_la_df['id_metric2']) & (pan_la_df['PbNab'] > 0) & ((pan_la_df['Gmod1'] >= 0) & (pan_la_df['Gmod2'] >= 0)) & (pan_la_df['Gmod1'] == pan_la_df['Gmod2']), 'pan_class'] = 3
    pan_la_df.loc[(pan_la_df['id_metric1'] == pan_la_df['id_metric2']) & (pan_la_df['PbNab'] > 0) & ((pan_la_df['Gmod1'] >= 0) & (pan_la_df['Gmod2'] >= 0)) & (pan_la_df['Gmod1'] == pan_la_df['Gmod2']), 'pan_class'] = 4

# Class 0 assignments are done later in the code


---
## Housekeeping
Account for the case when no anomaly is assigned to a class. This still allows for zero event anomalies to be reported.


In [ ]:
if panomaly_status:     
    if not len(pan_la_df[pan_la_df['pan_class'] > -1]): panomaly_status = False


---
## Calculate the final linkage evaluation metric
The final linkage evaluation metric, which denotes the relative "strength" or "importance" of each linked anomaly pair, is used to decide the True/False verdict for reporting the linkage. This metric, Qpan, is calculated slightly differently depending on the class of the linkage (see below). Future improvements to the calculation of Qpan based on experience could be made in the following cell.

In [ ]:
if panomaly_status: 
    # Scale factor are used to put the class 1 & 2 Qpan values on a comparable scale as those for class 3 & 4
    if (np.isnan(Qp_max)) | (np.isnan(Qg_max)) | (Qp_max == 0.0) | (Qp_max+Qg_max <= 0.0):
        Qp_scale_factor = 1.0
        Qg_scale_factor = 1.0
    else:
        Qp_scale_factor = (Qp_max/(Qp_max + Qg_max))
        Qg_scale_factor = (Qg_max/Qp_max)
        
    # Qpan for class 3 & 4 is calculated by combining Qp and Qg using the combine_scores method.
    # Qpan for class 1 & 2 is calculated by normalizing the non-zero metric (Qg or Qp, respectively) 
    # then multiplying by the scale factor.
    for nclass in pan_classes:
        if nclass == 4: 
            pan_la_df.at[pan_la_df['pan_class'] == nclass, 'Qpan'] = combine_scores(pan_la_df['Qp'], pan_la_df['Qg'])
        if nclass == 3: 
            pan_la_df.at[pan_la_df['pan_class'] == nclass, 'Qpan'] = combine_scores(pan_la_df['Qp'], pan_la_df['Qg'])
        if nclass == 2: 
            pan_la_df.at[pan_la_df['pan_class'] == nclass, 'Qg'] = -1
            pan_la_df.at[pan_la_df['pan_class'] == nclass, 'Qpan'] = normalize_to_max(pan_la_df['Qp'])*Qp_scale_factor
        if nclass == 1: 
            pan_la_df.at[pan_la_df['pan_class'] == nclass, 'Qp'] = -1
            pan_la_df.at[pan_la_df['pan_class'] == nclass, 'Qpan'] = normalize_to_max(pan_la_df['Qg'])*Qg_scale_factor


---
## Decide the verdict
The Qpan values are sent to verdict_function to decide whether or not a linkage should be surfaced. See the notes above in the verdict_function section.

In [ ]:
if panomaly_status: 
    for nclass in [4, 3]:
        if nclass == 4: 
            y1 = pan_la_df[pan_la_df['pan_class'] == nclass].Qpan
            y2 = pan_la_df[pan_la_df['pan_class'] == nclass].index
            method = 'all'
        if nclass == 3: 
            # this section also applies to class 2 and class 1, but these could be moved into their own sections
            y1 = pan_la_df[(pan_la_df['pan_class'] == 3) | (pan_la_df['pan_class'] == 2) | (pan_la_df['pan_class'] == 1)].Qpan
            y2 = pan_la_df[(pan_la_df['pan_class'] == 3) | (pan_la_df['pan_class'] == 2) | (pan_la_df['pan_class'] == 1)].index
            method = 'limit10'

        if len(y1):
            for x, i in zip(y1, y2):
                pan_Verdict = pan_verdict_function(x, y1, method)
                pan_la_df.at[(pan_la_df.index == i), 'Qcrit'] = pan_Verdict[0]
                pan_la_df.at[(pan_la_df.index == i), 'Verdict'] = pan_Verdict[1]


---
## Assemble the final reporting table

The following several cells assemble a dataframe ```pan_master_report_df``` containing the linked anomalies with ```True``` verdicts. The anomalies are sorted by class. They have also been combined into groups based on the principle "if A is linked to B, and B is linked to C, then A is linked to C". This allows linked anomalies to be reported in groups larger than pairs (although, in some cases, only a pair of anomalies is linked). This table also contains the event names for the anomalous metrics, which shows the event inference relationship between some anomalies.

In [ ]:
# Functions used to simplify the dataframe construction

# Pulls several columns into lists that will be recombined later
def extract_series(df, idx):
    temp_df = (df.loc[idx, ['Gmod1', 'id_anom1', 'id_metric1', 'metric_name1', 'Gmod2', 'id_anom2', 'id_metric2', 'metric_name2']])
    series1 = temp_df['id_anom1'].to_list()+temp_df['id_anom2'].to_list()
    series2 = temp_df['id_metric1'].to_list()+temp_df['id_metric2'].to_list()
    series3 = temp_df['metric_name1'].to_list()+temp_df['metric_name2'].to_list()
    return series1, series2, series3


# Creates a dictionary to add to a list of dictionaries used to create the master report dataframe
# The fastest way to make a dataframe row-by-row is to construct it from  list of dictionaries, 
# where each dictionary is one row of the dataframe
# https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe/17496530#17496530
def create_series_dict(nclass, class_tags, series1, series2, series3):
    return {'pan_class':nclass, 'pan_class_tag':class_tags[nclass], 'id_anom':list(series1), 'id_metric':list(series2), 'metric_name':list(series3)}


In [ ]:
pan_master_report_df = [] # Create placeholder in case panomaly_status = False

if panomaly_status: 
    nclass = 4 # Class being sorted in this cell
    # Pull out the class members into a working dataframe
    pan_working_df = pan_la_df[pan_la_df['pan_class'] == nclass].sort_values(['Qpan'], ascending=False)

    # Create a dictionary of lists for linked anomalies sorted into groups, which
    # will be used to create a dataframe that populates the master report dataframe.
    pan_series_dict_list = []
    for i in sorted(set(pan_working_df['id_anom1'])):
        idx = (sorted(set(pan_working_df[(pan_working_df['id_anom1'] == i) & (pan_working_df['id_anom2'] == i) & (pan_working_df['Verdict'] == True)].index.to_list())))
        if idx:
            pan_series1, pan_series2, pan_series3 = extract_series(pan_working_df, idx)
            pan_series1, pan_series2, pan_series3 = zip(*sorted(set(zip(pan_series1, pan_series2, pan_series3))))

            pan_series_dict = create_series_dict(nclass, pan_class_tags, pan_series1, pan_series2, pan_series3)
            pan_series_dict_list.append(pan_series_dict)

    # Create the master report dataframe
    if pan_series_dict_list:
        pan_series_df = pd.DataFrame(pan_series_dict_list)
        pan_master_report_df = pan_series_df.copy()

    if VERBOSE: 
        print('Created dataframe of anomalies/metrics linked to themselves (class = '+str(nclass)+')')
        display(pan_master_report_df)


In [ ]:
# Check for existence of pan_master_report_df and create placeholder if it doesn't exist already
if len(pan_master_report_df) < 1: pan_master_report_df = [] 

if panomaly_status: 
    nclass = 3 # Class being sorted in this cell
    # Pull out the class members into a working dataframe
    pan_working_df = pan_la_df[pan_la_df['pan_class'] == nclass].sort_values(['Qpan'], ascending=False)

    pan_series_dict_list = []
    pan_unique_modularity_classes = sorted(set(pan_working_df['Gmod1']))
    # Loop over modularity classes, since these define the Graph-based anomaly linkages
    for i in pan_unique_modularity_classes:
        idx = (sorted(set(pan_working_df[(pan_working_df['Gmod1'] == i) & (pan_working_df['Gmod2'] == i) & (pan_working_df['Verdict'] == True)].index.to_list())))
        if idx:
            pan_series1, pan_series2, pan_series3 = extract_series(pan_working_df, idx)
            pan_series1, pan_series2, pan_series3 = zip(*sorted(set(zip(pan_series1, pan_series2, pan_series3))))

            pan_series_dict = create_series_dict(nclass, pan_class_tags, pan_series1, pan_series2, pan_series3)
            pan_series_dict_list.append(pan_series_dict)

    # Create the master report dataframe if it doesn't exist, otherwise append to it
    if pan_series_dict_list:
        pan_series_df = pd.DataFrame(pan_series_dict_list)
        if len(pan_master_report_df) > 0:
            pan_master_report_df = pan_master_report_df.append(pan_series_df, ignore_index=True)
        else:
            pan_master_report_df = pan_series_df.copy()

    if VERBOSE: 
        print('Appended anomalies/metrics linked by Qp and Qg (class = '+str(nclass)+')')
        display(pan_master_report_df)


In [ ]:
# Check for existence of pan_master_report_df and create placeholder if it doesn't exist already
if len(pan_master_report_df) < 1: pan_master_report_df = []

if panomaly_status: 
    nclass = 2 # Class being sorted in this cell
    # Pull out the class members into a working dataframe
    pan_working_df = pan_la_df[pan_la_df['pan_class'] == nclass].sort_values(['Qp'], ascending=False)
    # There are a lot of Verdict=False transitions in class = 2, so let's avoid 
    # iterating over all of them by defining an iteration list of just Verdict=True
    pan_working_true_df = pan_la_df[(pan_la_df['pan_class'] == nclass) & (pan_la_df['Verdict'] == True)].sort_values(['Qp'], ascending=False)
    pan_working_true_df = pd.unique(pan_working_true_df[['id_anom1', 'id_anom2']].values.ravel('K'))

    pan_used_idx = []
    pan_series_dict_list = []
    for i in pan_working_true_df:
        idx0 = (sorted(set(pan_working_df[(pan_working_df['Qp'] > 0) & (pan_working_df['Gmod1'] != pan_working_df['Gmod2']) & (pan_working_df['Verdict'] == True)].index.to_list())))
        idx = list(set(idx0) - set(pan_used_idx))
        if idx:
            if not pan_used_idx:
                pan_used_idx = idx.copy()
            else:
                pan_used_idx = list(set(pan_used_idx.append(idx)))
            pan_series1, pan_series2, pan_series3 = extract_series(pan_working_df, idx)
            pan_series1, pan_series2, pan_series3 = zip(*sorted(set(zip(pan_series1, pan_series2, pan_series3))))

            pan_series_dict = create_series_dict(nclass, pan_class_tags, pan_series1, pan_series2, pan_series3)
            pan_series_dict_list.append(pan_series_dict)

    # Create the master report dataframe if it doesn't exist, otherwise append to it
    if pan_series_dict_list:
        pan_series_df = pd.DataFrame(pan_series_dict_list)
        if len(pan_master_report_df) > 0:
            pan_master_report_df = pan_master_report_df.append(pan_series_df, ignore_index=True)
        else:
            pan_master_report_df = pan_series_df.copy()

    if VERBOSE: 
        print('Appended anomalies/metrics linked by Qp only (class = '+str(nclass)+')')
        display(pan_master_report_df)
    

In [ ]:
# Check for existence of pan_master_report_df and create placeholder if it doesn't exist already
if len(pan_master_report_df) < 1: pan_master_report_df = []

if panomaly_status: 
    nclass = 1 # Class being sorted in this cell
    # Pull out the class members into a working dataframe
    pan_working_df = pan_la_df[pan_la_df['pan_class'] == nclass].sort_values(['Qg'], ascending=False)

    pan_series_dict_list = []
    pan_unique_modularity_classes = sorted(set(pan_working_df['Gmod1']))
    for i in pan_unique_modularity_classes:
        idx = (sorted(set(pan_working_df[(pan_working_df['Gmod1'] == i) & (pan_working_df['Gmod2'] == i) & (pan_working_df['Qp'] <= 0) & (pan_working_df['Verdict'] == True)].index.to_list())))
        if idx:
            pan_series1, pan_series2, pan_series3 = extract_series(pan_working_df, idx)
            pan_series1, pan_series2, pan_series3 = zip(*sorted(set(zip(pan_series1, pan_series2, pan_series3))))

            pan_series_dict = create_series_dict(nclass, pan_class_tags, pan_series1, pan_series2, pan_series3)
            pan_series_dict_list.append(pan_series_dict)

    # Create the master report dataframe if it doesn't exist, otherwise append to it
    if pan_series_dict_list:
        pan_series_df = pd.DataFrame(pan_series_dict_list)
        if len(pan_master_report_df) > 0:
            pan_master_report_df = pan_master_report_df.append(pan_series_df, ignore_index=True)
        else:
            pan_master_report_df = pan_series_df.copy()

    if VERBOSE: 
        print('Appended anomalies/metrics linked by Qg only (class = '+str(nclass)+')')
        display(pan_master_report_df)


In [ ]:
# Check for existence of pan_master_report_df and create placeholder if it doesn't exist already
if len(pan_master_report_df) < 1: pan_master_report_df = []

if pan_anom_zero_events: 
    nclass = 0 # Class being sorted in this cell

    for item in pan_anom_zero_events:
        if len(pan_la_df) > 0:
            pan_la_df.loc[(pan_la_df['id_metric1'] == item[1]) & (pan_la_df['id_metric2'] == item[1]) & (pan_la_df['N1'] == 0) & (pan_la_df['N2'] == 0), 'Class'] = 0
            pan_la_df.loc[(pan_la_df['id_metric1'] == item[1]) & (pan_la_df['id_metric2'] == item[1]) & (pan_la_df['N1'] == 0) & (pan_la_df['N2'] == 0), 'Qcrit'] = 0
            pan_la_df.loc[(pan_la_df['id_metric1'] == item[1]) & (pan_la_df['id_metric2'] == item[1]) & (pan_la_df['N1'] == 0) & (pan_la_df['N2'] == 0), 'Verdict'] = True
        
        pan_series_dict = create_series_dict(nclass, pan_class_tags, [item[0]], [item[1]], [item[2]])
        
        # Create the master report dataframe if it doesn't exist, otherwise append to it
        pan_series_df = pd.DataFrame(pan_series_dict)
        if len(pan_master_report_df) > 0:
            pan_master_report_df = pan_master_report_df.append(pan_series_df, ignore_index=True)
        else:
            pan_master_report_df = pan_series_df.copy()

    if VERBOSE: 
        print('Appended zero event anomalies (class = '+str(nclass)+')')
        display(pan_master_report_df)
    

In [ ]:
# This cell is mainly just for troubleshooting - it prints a convenient summary of information at the end of the run.
if VERBOSE:
    print('panomaly_status =', panomaly_status)
    print('pan_anom_zero_events =', pan_anom_zero_events)
    if panomaly_status:
        print('pan_time_start =', pan_time_start)
        if (not pan_anom_zero_events):
            print('org, app, users, nodes, anomalies: ', pan_org_id, pan_app_id, pan_nusers, pan_Nnodes, len(pan_anom_ids))
        else:
            print('org, app, users, nodes, anomalies: ', pan_org_id, pan_app_id, pan_nusers, len(pan_user_metrics_all), len(pan_anom_ids))

    display(pan_master_report_df)
    display(pan_la_df)
